# Developing of the pipeline

In [1]:
from data_loaders import get_data_loader, RotationWrapper
from utils.visualisation import showInRow

import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
import torchvision.models as models
from torchvision import transforms

seed_everything(123456)

123456

In [2]:
transform=transforms.Compose([#transforms.Grayscale(),
                              transforms.Resize((320,320)),
                              transforms.ToTensor(),
                              #transforms.Normalize((0.5,), (0.5,))
                             ])

In [9]:
class ResNet18_Rotation(pl.LightningModule):
    def __init__(self, num_classes, dataset_name, pretrained=True, transform=None, mode="pretrain", batch_size=16):
        # Load model
        super(ResNet18_Rotation, self).__init__()
        self.model = models.resnet18(pretrained=pretrained)
        self.mode = mode
        self.mode_outputs = {
            "pretrain": 4,
            "train": num_classes
        }
        self.loss = {
            "pretrain": nn.CrossEntropyLoss(),
            "train": nn.BCEWithLogitsLoss()
        }
        self.opt = {
            "pretrain": SGD(self.parameters(), lr=1e-4, momentum=0.9),
            "train": SGD(self.parameters(), lr=1e-4, momentum=0.9)
        }
        self.batch_size = batch_size
        self.dataset = dataset_name
        self.transform = transform
    
    
    def forward(self, x):
        x = self.model(x)
        return x
    
    def training_step(self, batch, batch_idx):
        if self.mode == "pretrain":
            images, labels = \
                batch['image'], batch['label']
        else:
            images, labels = \
                batch['image'], batch['one_hot_label']
        images = images.cuda()
        labels = labels.cuda()
        logits = self(images)
        loss = self.loss[self.mode](logits, labels)
        
        return loss
    
    def configure_optimizers(self):
        return self.opt[self.mode]
          
    
    
    def change_mode(self, mode):
        if mode in self.mode_outputs:
            self.mode = mode
            self.model.fc = nn.Linear(in_features=512, out_features=self.mode_outputs[self.mode], bias=True)
        else:
            print("Unknown mode, only next available: ", list(self.mode_outputs.keys()))
            
            
    def train_dataloader(self):
        if self.mode == "pretrain":
            train_raw = get_data_loader(self.dataset, part="train")
            train = RotationWrapper(train_raw, transform=self.transform)
        else:
            train = get_data_loader(self.dataset, part="train", transform=self.transform)    
        return DataLoader(train, batch_size=64, shuffle=True, num_workers=2)
    
    
    def val_dataloader(self):
        if self.mode == "pretrain":
            train_raw = get_data_loader(self.dataset, part="val")
            train = RotationWrapper(train_raw, transform=self.transform)
        else:
            train = get_data_loader(self.dataset, part="val", transform=self.transform)    
        return DataLoader(train, batch_size=self.batch_size, shuffle=False, num_workers=1)

    
    def test_dataloader(self):
        if self.mode == "pretrain":
            test_raw = get_data_loader(self.dataset, part="test")
            test = RotationWrapper(test_raw, transform=self.transform)
        else:
            test = get_data_loader(self.dataset, part="test", transform=self.transform)    
        return DataLoader(test, batch_size=self.batch_size, shuffle=False, num_workers=1)
    

In [14]:
rnet = ResNet18_Rotation(num_classes=15, dataset_name="Chest14", pretrained=True, 
                         transform=transform, mode="pretrain", batch_size=2)
if torch.cuda.is_available():
    rnet = rnet.cuda()
trainer = pl.Trainer(gpus=1, deterministic=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [15]:
trainer.fit(rnet)


  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/intern/.local/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/intern/.local/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/intern/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 74, in default_collate
    return {key: default_collate([d[key] for d in batch]) for key in elem}
  File "/home/intern/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 74, in <dictcomp>
    return {key: default_collate([d[key] for d in batch]) for key in elem}
  File "/home/intern/.local/lib/python3.7/site-packages/torch/utils/data/_utils/collate.py", line 53, in default_collate
    storage = elem.storage()._new_shared(numel)
  File "/home/intern/.local/lib/python3.7/